In [4]:
import pandas as pd
import numpy as np
from itertools import accumulate
from graphdatascience import GraphDataScience
pd.options.display.width = 0
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 1000)

# Prepare and Load Data
In this notebook we subsample, format and load data into Neo4j.  Our source will be the training dataset used in the [CIKM Cross-Device Entity Linking Challenge](https://competitions.codalab.org/competitions/11171). The raw dataset can be found [here](https://drive.google.com/file/d/0B7XZSACQf0Kdc3BnZzdEZzR6X0k/view?usp=sharing&resourcekey=0-KcizLV8DrV0jw_VLPNLvxQ). Before proceeding with the below, download the data, unzip and place in the `SOURCE_DATA_PATH` on the local machine.

## Directories and URL Settings
- `SOURCE_DATA_PATH` sets the data path on your machine where the source data is located.
- `NEO4J_IMPORT_DIR` sets the the location to write formatted csvs to
- `NEO4j_IMPORT_URL` sets the URL for Neo4j to use during loading.

- For __on-prem installations__ File locations for each OS are listed in the [Neo4j Operations Manual](https://neo4j.com/docs/operations-manual/current/configuration/file-locations). We recommend putting files in Neo4j’s `import` directory, as it keeps the environment secure. When you do this, you can use 'file://' for `NEO4j_IMPORT_URL`. If, however, you need to access files in other locations, you can adjust configurations for this per directions in our [documentation](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/#query-load-csv-introduction).
- For __AuraDS__ you can load csvs from FTP, HTTP, or HTTPS servers. You would adjust this script to write to to specified location and update `NEO4j_IMPORT_URL` accordingly.

The below example is for on-prem, specifically Neo4j desktop, but can be adjusted for other Neo4j deployments.

While you can load data via the Python driver, loading data from formatted csvs file with `LOAD CSV` will be significantly faster.


In [5]:
SOURCE_DATA_PATH = './data'
NEO4J_IMPORT_DIR = '/Users/zachblumenfeld/Library/Application Support/Neo4j Desktop/Application/relate-data/' + \
                   'dbmss/dbms-2a54a0cf-88ac-43b5-9a2b-9bbb82c8c75a/import'
NEO4j_IMPORT_URL = 'file://'

## Prepare Data
To prepare the data, we subsample it based on the `SAMPLE_RATE` defined below and transforms it into five input data sets.

1. __(Users)__: Nodes Representing Users


2. __(Users)-\[SAME_AS\]->(Users)__: Relationships representing labeled Entity-Linkage/Alignments between Users.  Will be used for supervised link prediction.


3. __(Websites)__: Nodes representing websites. Each URL path from the input dataset will be decomposed into multiple websites to represent the path hierarchy. The depth to consider for the hierarchy is set by the `MAX_URL_SEGMENTS` variable below. Each URL will be split into `MAX_URL_SEGMENTS + 1` website nodes where `MAX_URL_SEGMENTS` website nodes will be used for the hierarchy and the last website node will correspond to the the full url path.  For example, if `MAX_URL_SEGMENTS=3`, then the URL `a/b/c/d/e` will become 4 website nodes identified by the urls `[a, a/b, a/b/c, a/b/c/d/e]` while the website `aa/bb/cc` will become `[aa, aa/bb, aa/bb/cc, aa/bb/cc]`. The website nodes will be deduplicated so each unique url in the hierarchy is only represented by a singe node in the graph. This is done because the full url paths are fairly unique and provide only sparse connectivity between users.  Splitting urls in this way allows us to draw meaningful relationships from which we can engineer features for link prediction.


4. __(User)-\[VISITED\]->(Website)__: Relationships representing users interacting with websites. The relationships will be weighted by number of visits.


5. __(Website)-\[CHILD_OF\]->(Website)__: Relationships representing hierarchical structure of websites. i.e. if the URL `a/b/c/d` is split into `[a, a/b, a/b/c, a/b/c/d]` per above rules, then there would be a `CHILD_OF` relationship going from `a/b/c/d -> a/b/c`, `a/b/c -> a/b`, and `a/b -> a`.  This allows for a much more well connected graph.

A larger subsample or the complete dataset can be used. Just keep in mind that the larger the dataset, the more memory will be needed for GDS.

### Parameters for Sampling and Transformation

In [6]:
# The rate between(0,1) to sample User Ids from facts.json.
# Recommend using <=0.1 for quick experimentation
SAMPLE_RATE = 0.1

# Whether to sample only pairs of nodes that are aligned in the training set.
# This will bias the graph to a higher true class ratio if set to True
ONLY_SAMPLE_ALIGNED_PAIRS = False

# The number of URL segments to consider for the URLs
MAX_URL_SEGMENTS = 5

# The Random Seed to use throughout
RANDOM_SEED = 7474

### Format Facts (a.k.a Logged Events)

We load facts.json to sample from it and shape a unique __userid->eventid__ data frame.
The size of the data set may limit the ability to load it all at once depending on your environment so the below example iterates over the json file in chunks.  Feel free to tune the paramters so it is optimized for your machine.

In [7]:
#helper functions
def extract_values_from_facts(row):
    return row.facts['fid'], row.facts['ts']

def format_facts(raw_df):
    df = raw_df.explode("facts")
    df[['fid','ts']] = df.apply(lambda row: extract_values_from_facts(row), axis=1, result_type ='expand')
    #df.drop(columns = ["facts"], inplace=True)
    return df

In [10]:
# Labeled Entity-Linkages/Alignments
raw_train_df = pd.read_csv(f'{SOURCE_DATA_PATH}/data-train-dca/train.csv', header=0, names=['uid1', 'uid2'])

In [11]:
#iterative sampling of facts.json
n = 0
facts_dfs = []
chunk_size = 5000
if ONLY_SAMPLE_ALIGNED_PAIRS:
    sample_train_df = raw_train_df.sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
    unique_aligned_uids = pd.concat([sample_train_df.uid1, sample_train_df.uid2], ignore_index=True) \
        .drop_duplicates().tolist()
    with pd.read_json(f'{SOURCE_DATA_PATH}/data-train-dca/facts.json', lines=True, chunksize=chunk_size) as reader:
        for chunk_df in reader:
            is_aligned_indicator = chunk_df.uid.isin(unique_aligned_uids)
            # join on alignments present, sub-sample, and format
            facts_dfs.append(format_facts(chunk_df[is_aligned_indicator]))
            #print progress
            n = n + chunk_size
            # print(n)
else:
    unique_aligned_uids = pd.concat([raw_train_df.uid1, raw_train_df.uid2], ignore_index=True) \
        .drop_duplicates().tolist()
    with pd.read_json(f'{SOURCE_DATA_PATH}/data-train-dca/facts.json', lines=True, chunksize=chunk_size) as reader:
        for chunk_df in reader:
            is_aligned_indicator = chunk_df.uid.isin(unique_aligned_uids)
            # join on alignments present, sub-sample, and format
            chunk_df_aligned = chunk_df[is_aligned_indicator].sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
            facts_dfs.append(format_facts(chunk_df_aligned ))
            # anti-join on alignments present
            chunk_df_not_aligned = chunk_df[~is_aligned_indicator].sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
            facts_dfs.append(format_facts(chunk_df_not_aligned ))
            #print progress
            n = n + chunk_size
            # print(n)

In [12]:
facts_df = pd.concat(facts_dfs, ignore_index=True)
facts_df.shape

(6739095, 4)

In [13]:
clean_facts_df = facts_df.drop_duplicates(subset = ["uid", "fid", "ts"])
clean_facts_df

,facts,uid,fid,ts
0,"{'fid': 6135636, 'ts': 1464201356667}",8478e6a4211f9cc38608ce07375ab587,6135636,1464201356667
1,"{'fid': 6135636, 'ts': 1464201326667}",8478e6a4211f9cc38608ce07375ab587,6135636,1464201326667
2,"{'fid': 10225497, 'ts': 1464193434239}",8478e6a4211f9cc38608ce07375ab587,10225497,1464193434239
3,"{'fid': 2036842, 'ts': 1464192577609}",8478e6a4211f9cc38608ce07375ab587,2036842,1464192577609
4,"{'fid': 12576917, 'ts': 1464192568454}",8478e6a4211f9cc38608ce07375ab587,12576917,1464192568454
...,...,...,...,...
6739090,"{'fid': 6570813, 'ts': 1465828799392}",14e6a05f05a4226b99fb86e0bc50dfad,6570813,1465828799392
6739091,"{'fid': 12854313, 'ts': 1465828788160}",14e6a05f05a4226b99fb86e0bc50dfad,12854313,1465828788160
6739092,"{'fid': 6056513, 'ts': 1465828469495}",14e6a05f05a4226b99fb86e0bc50dfad,6056513,1465828469495
6739093,"{'fid': 3335637, 'ts': 1465828089349}",14e6a05f05a4226b99fb86e0bc50dfad,3335637,1465828089349


### Subsample Training Alignments based on Facts Subsampling

In [15]:
unique_uid_df = clean_facts_df.uid.drop_duplicates().to_frame(name="uid")
## merge on both alignment columns
alignment_df = raw_train_df.merge(unique_uid_df, left_on="uid1", right_on="uid") \
    .merge(unique_uid_df, left_on="uid2", right_on="uid")

### Join URLS to Capture Site Visits

In [16]:
urls_df = pd.read_csv(f'{SOURCE_DATA_PATH}/data-train-dca/urls.csv', header=0, names=['fid', 'url'])
#urls_df

In [17]:
# Note that neither fid nor url have duplicate values
print(urls_df.fid.duplicated().sum())
print(urls_df.url.duplicated().sum())

0
0


In [18]:
# merge facts and url visits on fid.
user_web_df = clean_facts_df.merge(urls_df, on="fid")

In [19]:
# split the urls out by path segments so there are separate columns for the path hierarchy
def agg_path(s1, s2):
    return s1 + '/' + s2

def url_parse(url):
    query_pos = url.find('?')
    if query_pos != -1:
        url = url[0:query_pos]
    url_path = url.split('/')

    url_path_length = len(url_path)

    if url_path_length>=MAX_URL_SEGMENTS:
        url_agg_paths = list(accumulate(url_path[0:MAX_URL_SEGMENTS], agg_path))
    else:
        url_agg_paths_staging = list(accumulate(url_path[0:url_path_length], agg_path))
        url_agg_paths = url_agg_paths_staging + [url_agg_paths_staging[-1]] * (MAX_URL_SEGMENTS - url_path_length)

    return  url_agg_paths + [url_path_length]

url_segment_names = ["url_segment_" + str(x) for x in np.arange(0, MAX_URL_SEGMENTS, 1)]
url_new_column_names = url_segment_names + ["url_path_depth"]

In [20]:
user_web_df[url_new_column_names] = \
    user_web_df.apply(lambda row: url_parse(row.url), axis=1, result_type ='expand')
user_web_df

,facts,uid,fid,ts,url,url_segment_0,url_segment_1,url_segment_2,url_segment_3,url_segment_4,url_path_depth
0,"{'fid': 6135636, 'ts': 1464201356667}",8478e6a4211f9cc38608ce07375ab587,6135636,1464201356667,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,d33de24430bf4005,d33de24430bf4005/7de2f069da3a3655,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,4
1,"{'fid': 6135636, 'ts': 1464201326667}",8478e6a4211f9cc38608ce07375ab587,6135636,1464201326667,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,d33de24430bf4005,d33de24430bf4005/7de2f069da3a3655,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,d33de24430bf4005/7de2f069da3a3655/6a30a933225c3548/a9da5e44f08b302e,4
2,"{'fid': 10225497, 'ts': 1464193434239}",8478e6a4211f9cc38608ce07375ab587,10225497,1464193434239,8992698f779504dd/64dde630ecabd85/d13353338202e7d2/879271064ca0b858/c763353373b99001,8992698f779504dd,8992698f779504dd/64dde630ecabd85,8992698f779504dd/64dde630ecabd85/d13353338202e7d2,8992698f779504dd/64dde630ecabd85/d13353338202e7d2/879271064ca0b858,8992698f779504dd/64dde630ecabd85/d13353338202e7d2/879271064ca0b858/c763353373b99001,5
3,"{'fid': 2036842, 'ts': 1464192577609}",8478e6a4211f9cc38608ce07375ab587,2036842,1464192577609,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787,0e88b67f68cd2787/537df06f127af450,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,3
4,"{'fid': 2036842, 'ts': 1465538647609}",ff470caf31d1322878f5bab93fe87008,2036842,1465538647609,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787,0e88b67f68cd2787/537df06f127af450,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,0e88b67f68cd2787/537df06f127af450/3acb09862ed13756,3
...,...,...,...,...,...,...,...,...,...,...,...
6739090,"{'fid': 12914313, 'ts': 1465828854927}",14e6a05f05a4226b99fb86e0bc50dfad,12914313,1465828854927,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/3b329b665c0bbe1d,ed95a9a5be30e4c8,ed95a9a5be30e4c8/d400c9e183de73f3,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/3b329b665c0bbe1d,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/3b329b665c0bbe1d,4
6739091,"{'fid': 13683932, 'ts': 1465828848893}",14e6a05f05a4226b99fb86e0bc50dfad,13683932,1465828848893,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/b925e91110fa7167,ed95a9a5be30e4c8,ed95a9a5be30e4c8/d400c9e183de73f3,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/b925e91110fa7167,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/b925e91110fa7167,4
6739092,"{'fid': 12854313, 'ts': 1465828788160}",14e6a05f05a4226b99fb86e0bc50dfad,12854313,1465828788160,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?32888c1b78217f8d,ed95a9a5be30e4c8,ed95a9a5be30e4c8/d400c9e183de73f3,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc,5
6739093,"{'fid': 6056513, 'ts': 1465828469495}",14e6a05f05a4226b99fb86e0bc50dfad,6056513,1465828469495,f3185e90acedd8c/6948a094778c660/fb6015ae52a94e92/2ba3d4bb006f0afd/e24ac23d52ec7bcc,f3185e90acedd8c,f3185e90acedd8c/6948a094778c660,f3185e90acedd8c/6948a094778c660/fb6015ae52a94e92,f3185e90acedd8c/6948a094778c660/fb6015ae52a94e92/2ba3d4bb006f0afd,f3185e90acedd8c/6948a094778c660/fb6015ae52a94e92/2ba3d4bb006f0afd/e24ac23d52ec7bcc,5


### Format and Write CSVs

In [21]:
# Users
user_df = user_web_df['uid'].drop_duplicates()
user_df.to_csv(f'{NEO4J_IMPORT_DIR}/users.csv', index=False)
user_df

0          8478e6a4211f9cc38608ce07375ab587
4          ff470caf31d1322878f5bab93fe87008
8          cbc8e6ce72ece53140217ff26f1ac327
9          ff6ba95c715bd148a534b5ac5a5a00cd
10         3950e76c6196e4b970b54bca187ae8d3
                         ...               
6344115    e1be657266ee6ae9ceeeeb13b214ceb2
6488303    93745acc93d3745aa0e334a8f1e2428c
6554604    9fdb0985476c62514ad63250ae7d5421
6602227    524b9360faf628eae42d2f45d9b9be8c
6634982    4dedc44158b262ef50b45d3df2cecf2d
Name: uid, Length: 33941, dtype: object

In [22]:
# (User)-[SAME_AS]->(User)
alignment_df.to_csv(f'{NEO4J_IMPORT_DIR}/user_alignments.csv', index=False)
alignment_df

,uid1,uid2,uid_x,uid_y
0,ab301fd4824c32f1d2946ede295e2e42,b669962d925feb0d9f1ab3e166684458,ab301fd4824c32f1d2946ede295e2e42,b669962d925feb0d9f1ab3e166684458
1,3662c746e4bde44c877a43df203b35d2,c249aaae2888209681b0427f847795a1,3662c746e4bde44c877a43df203b35d2,c249aaae2888209681b0427f847795a1
2,15b68e3803dd57699081a7d032ed4771,f185649bf662c60d1c6db9776015a053,15b68e3803dd57699081a7d032ed4771,f185649bf662c60d1c6db9776015a053
3,1e3ac11c9915011ecb409d49030ed723,f67853bc71451337c60ce96b11731b1b,1e3ac11c9915011ecb409d49030ed723,f67853bc71451337c60ce96b11731b1b
4,9a45e25885b188a1376b9d36fdbe6f66,a210c77b3972ef0e38a60383e3d643d7,9a45e25885b188a1376b9d36fdbe6f66,a210c77b3972ef0e38a60383e3d643d7
...,...,...,...,...
5004,28d676ec00fddd57beee374a234bee13,2cf039cd841c9cf4d352f1247b343e49,28d676ec00fddd57beee374a234bee13,2cf039cd841c9cf4d352f1247b343e49
5005,29c9210c684ff1824ba50302ae6b48dc,9cd3d95790e17738d02f0b6a87f368ae,29c9210c684ff1824ba50302ae6b48dc,9cd3d95790e17738d02f0b6a87f368ae
5006,29c9210c684ff1824ba50302ae6b48dc,c9da17dd14b1760345693cd61d91edbb,29c9210c684ff1824ba50302ae6b48dc,c9da17dd14b1760345693cd61d91edbb
5007,9cd3d95790e17738d02f0b6a87f368ae,c9da17dd14b1760345693cd61d91edbb,9cd3d95790e17738d02f0b6a87f368ae,c9da17dd14b1760345693cd61d91edbb


In [23]:
# (Website)-[:CHILD_OF]->(Website)
all_url_column_names = url_segment_names + ['url']

web_hierarchy_dfs = []

for i in range(len(all_url_column_names)-1):
    url1 = all_url_column_names[i]
    url2 = all_url_column_names[i + 1]

    web_hier_segment_df = user_web_df[[url1, url2]]
    web_hier_segment_df = web_hier_segment_df.rename(columns={url1:"url1", url2:"url2"})
    web_hier_segment_df = web_hier_segment_df[web_hier_segment_df.url1 != web_hier_segment_df.url2]
    web_hierarchy_dfs.append(web_hier_segment_df)

web_hierarchy_df = pd.concat(web_hierarchy_dfs, ignore_index=True).drop_duplicates()

web_hierarchy_df.to_csv(f'{NEO4J_IMPORT_DIR}/web_hierarchy.csv', index=False)
web_hierarchy_df

,url1,url2
0,d33de24430bf4005,d33de24430bf4005/7de2f069da3a3655
2,8992698f779504dd,8992698f779504dd/64dde630ecabd85
3,0e88b67f68cd2787,0e88b67f68cd2787/537df06f127af450
143,179e7eb2c28811df,179e7eb2c28811df/a828c5625a5d322
145,657b15c5499c303b,657b15c5499c303b/8601f09e63f92ad6
...,...,...
15965112,d60f4f5ba631d8df/52021da41b4f622/227c5d7bda6f3e88,d60f4f5ba631d8df/52021da41b4f622/227c5d7bda6f3e88?f32ed304ed5d1e4f
15965113,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?826be01e660d62ad
15965114,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?e9620fa84042e313
15965115,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?0c735d45149a776


In [24]:
# Websites
websites_dfs = []
for url in all_url_column_names:
    websites_df_staging = user_web_df[[url, "url_path_depth"]]
    if url != 'url':
        websites_df_staging = websites_df_staging.rename(columns={url:"url"})
        websites_df_staging = websites_df_staging.drop(columns="url_path_depth")
        websites_df_staging = websites_df_staging.assign(url_path_depth=int(url[-1]) + 1)
    websites_dfs.append(websites_df_staging)

websites_df = pd.concat(websites_dfs, ignore_index=True).drop_duplicates(subset="url")
websites_df.to_csv(f'{NEO4J_IMPORT_DIR}/websites.csv', index=False)
websites_df

,url,url_path_depth
0,d33de24430bf4005,1
2,8992698f779504dd,1
3,0e88b67f68cd2787,1
143,179e7eb2c28811df,1
145,657b15c5499c303b,1
...,...,...
40434540,d60f4f5ba631d8df/52021da41b4f622/227c5d7bda6f3e88?f32ed304ed5d1e4f,3
40434562,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?826be01e660d62ad,5
40434563,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?e9620fa84042e313,5
40434564,ed95a9a5be30e4c8/d400c9e183de73f3/761fabc659e8b0ea/da0730aa0b149bd0/ee44da7e5dd2b3cc?0c735d45149a776,5


In [25]:
## (User)-[:VISITED]->(Website)
def sorted_list(x):
    res = x.tolist()
    res.sort()
    return res

users_web_visits_df = user_web_df[['uid','url','facts','fid','ts']].groupby(['uid','url']) \
    .agg({'facts': lambda x: x.tolist(),'fid': lambda x: set(x.tolist()), 'ts': lambda x: sorted_list(x)}) \
    .reset_index()
users_web_visits_df['number_of_visits'] = users_web_visits_df.fid.apply(lambda x: len(x))
users_web_visits_df.to_csv(f'{NEO4J_IMPORT_DIR}/user_website_visits.csv', index=False)
users_web_visits_df

,uid,url,facts,fid,ts,number_of_visits
0,0001cd76a7fb602d8a8117d6a9da149e,17446fcdba10b6d8/45c5a7a4c893f6a1/dc47363579806396/ccd00814d9e8747,"[{'fid': 6556998, 'ts': 1464822828629}]",{6556998},[1464822828629],1
1,0001cd76a7fb602d8a8117d6a9da149e,5e1208241ce0cc1f/6a30a933225c3548/58f9792b4821ccc0/c83355c3b2a0df6d,"[{'fid': 5012705, 'ts': 1464967923421}]",{5012705},[1464967923421],1
2,0001cd76a7fb602d8a8117d6a9da149e,6ae12ea8ec730ba5/281bb5a0f4846ea7/802fc6a2d4f41295/34702b07a20db/8b84b6e138385d6,"[{'fid': 4840746, 'ts': 1464821234838}, {'fid': 4840746, 'ts': 1464821201771}]",{4840746},"[1464821201771, 1464821234838]",1
3,0001cd76a7fb602d8a8117d6a9da149e,a2479ee9a8f008fb/f34f54ecea422c5f?f8198452150730be,"[{'fid': 13369326, 'ts': 1464823009674}]",{13369326},[1464823009674],1
4,0001cd76a7fb602d8a8117d6a9da149e,a997482113271d8f/5758f309e11931ce,"[{'fid': 8799201, 'ts': 1464822784110}]",{8799201},[1464822784110],1
...,...,...,...,...,...,...
2841386,fffe8dba19178cf592196ce09074c3e8,f51d84644e93caf1/c1e7c35981fef9da/aed120160eb27b8a/2da79da3ae8a5a0,"[{'fid': 12529937, 'ts': 1465231609021}, {'fid': 12529937, 'ts': 1465231579021}]",{12529937},"[1465231579021, 1465231609021]",1
2841387,fffe8dba19178cf592196ce09074c3e8,f51d84644e93caf1/c1e7c35981fef9da/aed120160eb27b8a/dd4daec06615b495?73e599f6763e4f17,"[{'fid': 6407588, 'ts': 1465231678041}, {'fid': 6407588, 'ts': 1465231648041}]",{6407588},"[1465231648041, 1465231678041]",1
2841388,fffe8dba19178cf592196ce09074c3e8,f51d84644e93caf1/c1e7c35981fef9da/c0995aab09f322e2/0ef0b6193d63a6f8,"[{'fid': 10158342, 'ts': 1465231686977}]",{10158342},[1465231686977],1
2841389,fffe8dba19178cf592196ce09074c3e8,f51d84644e93caf1/c1e7c35981fef9da/c0995aab09f322e2/18c74de9312d1d4c,"[{'fid': 6051312, 'ts': 1465231572885}]",{6051312},[1465231572885],1


## Load Data Into Neo4j
In this section we will load the above transformed data into Neo4j.

In [26]:
HOST = 'neo4j://localhost'
USERNAME = 'neo4j'
PASSWORD = 'password'

gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=False)

### Set Uniqueness Constraints
This step is critical for performant loading with merge patterns

In [27]:
gds.run_cypher('CREATE CONSTRAINT userId_unique IF NOT EXISTS ON (user:User) ASSERT user.userId  IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT url_unique IF NOT EXISTS ON (website:Website) ASSERT website.url IS UNIQUE')

""


### Load Nodes

In [28]:
#load users
gds.run_cypher(f'''
    LOAD CSV WITH HEADERS FROM '{NEO4j_IMPORT_URL}/users.csv' AS row
    MERGE (u:User {{userId:row.uid}})
    RETURN count(u);
''')

,count(u)
0,33941


In [30]:
#load websites
gds.run_cypher(f'''
    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM '{NEO4j_IMPORT_URL}/websites.csv' AS row
    MERGE (w:Website {{url: row.url}})
    ON CREATE
        SET w.urlPathDepth = toInteger(row.url_path_depth)
    RETURN count(w);
''')

,count(w)
0,2399920


### Load Relationships

In [31]:
# load ground truth identity alignments
gds.run_cypher(f'''
    LOAD CSV WITH HEADERS FROM '{NEO4j_IMPORT_URL}/user_alignments.csv' AS row
    MATCH(u1:User {{userId:row.uid1}})
    MATCH(u2:User {{userId:row.uid2}})
    MERGE (u1)-[r:SAME_AS]->(u2)
    RETURN count(r);
''')

,count(r)
0,5009


In [33]:
# load website hierarchy
gds.run_cypher(f'''
    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM '{NEO4j_IMPORT_URL}/web_hierarchy.csv' AS row
    MATCH(w1:Website {{url: row.url1}})
    MATCH(w2:Website {{url: row.url2}})
    MERGE (w2)-[r:CHILD_OF]->(w1)
    ON CREATE
        SET r.weight = toInteger(w1.urlPathDepth)
    RETURN count(r);
''')

,count(r)
0,2330404


In [34]:
# load visits
gds.run_cypher(f'''
    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM '{NEO4j_IMPORT_URL}/user_website_visits.csv' AS row
    MATCH(u:User {{userId:row.uid}})
    MATCH(w:Website {{url: row.url}})
    WITH u, w, row, toInteger(row.number_of_visits) as num_visits
    MERGE (u)-[r:VISITED]->(w)
    ON CREATE
      SET r.eventIds = split(replace(replace(row.fid,'{{',''),'}}',''),', ')
      SET r.timeStamps = reduce(dateTimes = [], e in split(replace(replace(row.ts,'[',''),']',''),', ') |
        dateTimes + datetime({{epochMillis:toInteger(e)}}))
      SET r.numberOfVisits = num_visits
      SET r.weight = num_visits * w.urlPathDepth
    RETURN count(r);
''')

,count(r)
0,2841391
